# Create a CHIRPS Rainfall Grid Stack in Africa

#### Table of Contents

1.  <a href="#intro_met">Introduction</a> <br>
2.  <a href="#specify_basin">Specify a River Basin</a> <br>
4.  <a href="#CHIRPS_to_RTS">Create CHIRPS Grid Stack</a> <br>

<!-- Hyperlink IDs must be unique to work in Jupyter Lab when
there are multiple notebooks open. Hence the trailing letter. -->

## Introduction <a id="intro_met"></a>

<b>Note:</b> This capability has been mostly delegated to the MINT data transformation team, since downloading all of the required files can take hours to days.

However, assuming all required files have already been downloaded to your computer it can also be performed using TopoFlow's <b>regrid</b> utility.  The specific function for this is called: <b>create_rts_from_nc_files()</b>.

For the [<b>MINT Project</b>](http://mint-project.info/), we have used three different
<b>global</b>, remote-sensing based products for rainfall, called:
<b>GPM</b>, <b>GLDAS</b>, and <b>CHIRPS</b>.  This notebook is focused on CHIRPS data.

In addition to space-time rainfall rates, some of the TopoFlow components can make use of other meteorological variables such as:  air temperature, soil temperature, relative humidity, surface wind speed, shortware radiation and longwave radiation. 
The <b>create_rts_from_nc_files()</b> function has a <b>var_name</b> argument and other settings to accommodate these additional variables.

## Import the TopoFlow regrid Utility

In [1]:
import os
from topoflow.utils import regrid

Importing TopoFlow 3.6 package modules:
   topoflow.utils
   topoflow.utils.tests
   topoflow.components
   topoflow.components.tests
   topoflow.framework
   topoflow.framework.tests
 


## Specify a River Basin  <a id="specify_basin"></a>

In [4]:

output_dir  = '/Users/peckhams/Downloads/'
tf_data_dir = '/Users/peckhams/TF_Data/'

# Directory with CHIRPS data for all of Africa
# time_span   = '2015-10_to_2018-10'
# chirps_dir  = tf_data_dir + 'CHIRPS_Rain_Africa_6hr_2015-10_to_2018-10_onedir/'  # All of Africa

time_span   = '2015-10_to_2020-06'
chirps_dir  = tf_data_dir + 'CHIRPS_Rain_Africa_6hr_2015-10_to_2020-06_onedir/'  # All of Africa

#site_prefix = 'Tana_120sec'     # (Kenya)
site_prefix = 'Galana_120sec'   # (Kenya)
#site_prefix = 'Baro-Gam_60sec'  # (Ethiopia)

if (site_prefix == 'Tana_120sec'):
    #----------------------------------------
    # DEM information for Tana River, Kenya
    #----------------------------------------
    short_name   = 'Tana'
    DEM_xres_sec = 120.0  # [arcseconds]
    DEM_yres_sec = 120.0
    DEM_ncols    = 131
    DEM_nrows    = 115
    minlat = -3.143333333333    # (south)   # -3.15 ??
    maxlat = 0.690              # (north)
    minlon = 36.40              # (west)
    maxlon = 40.766666666667    # (east)    # 40.75 ??
    
if (site_prefix == 'Galana_120sec'):
    #------------------------------------------
    # DEM information for Galana River, Kenya
    #------------------------------------------
    short_name   = 'Galana'
    DEM_xres_sec = 120.0  # [arcseconds]
    DEM_yres_sec = 120.0
    DEM_ncols    = 118
    DEM_nrows    = 79
    minlat = -3.573333333333    # (south)
    maxlat = -0.940             # (north)
    minlon = 36.30              # (west)
    maxlon = 40.233333333333    # (east)

if (site_prefix == 'Baro-Gam_60sec'):
    #--------------------------------------------
    # DEM information for Baro River, Ethiopia
    #--------------------------------------------
    short_name   = 'Baro'
    DEM_xres_sec = 60.0  # [arcseconds]
    DEM_yres_sec = 60.0
    DEM_ncols    = 134
    DEM_nrows    = 129
    minlat = 7.362083333332    # (south)
    maxlat = 9.503749999999    # (north)
    minlon = 34.221249999999   # (west)
    maxlon = 36.450416666666   # (east)
    
#--------------------
# Set the DEM bounds
#--------------------
DEM_bounds = [minlon, minlat, maxlon, maxlat]


## Create CHIRPS Grid Stack  <a id="CHIRPS_to_RTS"></a>

Here we use the CHIRPS Africa dataset, and a function in the TopoFlow <b>regrid</b>
utility called <b>create_rts_from_chirps_files()</b>.  This function is currently
hardwired to use the CHIRPS Africa dataset, which has the following georeferencing
information.

    #-----------------------------------    
    # CHIRPS Africa bounding box, etc.
    #-----------------------------------
    # chirps_minlat = -40.05
    # chirps_maxlat = 40.05
    # chirps_minlon = -20.05
    # chirps_maxlon = 55.05
    chirps_ulx      = -20.05
    chirps_uly      = 40.05
    chirps_xres_deg = 0.1
    chirps_yres_deg = -0.1     #####   NEED NEGATIVE HERE
    chirps_ncols    = 751
    chirps_nrows    = 801
    chirps_dtype    = 'float32'
    chirps_nodata   = -1.0
    
The website for downloading CHIRPS 2.0 Africa 6-hourly data is:
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_6-hourly/p1_bin/

To download a large number of files, it is convenient to use an FTP application such as Cyberduck.  The FTP address is:  ftp://data.chc.ucsb.udu, and you can set then username to "anonymous".<br>
Once connected, the files for 1981 to 2015 can be found in: <br>
&nbsp; &nbsp; &nbsp; &nbsp; pub/org/chc/products/CHIRPS-2.0/africa_6-hourly/p1_bin <br>
and the files for 2016 to 2021 in:<br>
&nbsp; &nbsp; &nbsp; &nbsp; pub/org/chc/products/CHIRPS-2.0/africa_6-hourly/p1_bin/extra_step <br>
There is one file for each 6-hour period, and each folder contains all of the files for a given year and month.
Cyberduck allows you to select multiple folders. You can then right click and choose "Download To..." to download to a specific folder.


In [5]:
rts_file = output_dir + 'CHIRPS_Rain_' + site_prefix + '_' + time_span + '.rts'
# rts_file = output_dir + 'CHIRPS_Rain_' + short_name + '_' + time_span + '.rts'

resample_algo = 'bilinear'
## resample_algo = 'nearest'
time_interval_hours = 6.0      #### For the 6-hourly rainfall product

os.chdir( chirps_dir )
regrid.create_rts_from_chirps_files( rts_file=rts_file,
           time_interval_hours=time_interval_hours,
           resample_algo=resample_algo,
           VERBOSE=False, DEM_bounds=DEM_bounds,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows )

Working...

chirps_file = rfe_gdas.bin.2015100100
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 1

chirps_file = rfe_gdas.bin.2015100106
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 2

chirps_file = rfe_gdas.bin.2015100112
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 3

chirps_file = rfe_gdas.bin.2015100118
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 4

chirps_file = rfe_gdas.bin.2015100200
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 5

chirps_file = rfe_gdas.bin.2015100206
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 6

chirps_file = rfe_gdas.bin.2015100212
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 7

chirps_file = rfe_gdas.bin.2015100218
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count =